# <div style="text-align: center">HashTag Generator for Social Media Posts</div>

![Multiple Hashtags for same theme](./images/hashtag-blog.png)



<div style="text-align: right">Sai Nikhil Dogiparty(dogiparty.s@husky.neu.edu)</div>
<div style="text-align: right"> Rohan Jahagirdar(jahagirdar.ro@husky.neu.edu)</div>

## Introduction
***

This research is about recommending Hashtags for a post on Social Networking Sites (like Twitter, etc) by processing the contents of the post, user's profile information and the large amount of data available from such Social Media Services. We analyse the post and predict the hashtags that match the contents of the post.



### #HashTag
#### Why is it so important?
With the growing influence of microblogging services, the number of users on these platforms has exponentially increased. 
The growing content on these platforms present an interesting problem for finding posts with a specific theme or content. To make this lookup easy, social media uses meta tagging by using hashtags (#). Using hashtags with social network posts lets researchers group similar themed user posts together and analyse them.

#### What are the problems we look to solve
This form of dynamic tagging by users sometimes creates multiple hashtags for similar theme which spreads the data across different tags making it difficult to aggregate posts and analyse posts that share same theme. 

![Multiple Hashtags for same theme](./images/InternationalWomensDayTags.jpeg)

From the perspective of users of such services, comming up with Hashtags that best match the post is an additional overhead. 

In this work we instead leverage the wealth of information available from users: namely the posts and other meta data information. By building a knowledge graph using machine learning algorithms, we predict hashtags for a post and recommend likeminded users to follow. While the approach is platform independent, we'll look at Twitter for this research.




#### Our approach to the solution
We explore two ways of combining these heterogeneous features into a learning framework. We'll discuss the approach in detail below:
1. Finding the closest hashtag to a user’s post using [word embeddings](https://en.wikipedia.org/wiki/Word_embedding) with [Word2vec](https://en.wikipedia.org/wiki/Word2vec) Models
2. Finding similar posts and users using [SpaCy](https://spacy.io/)- A Natural Language Processing(NLP) Neural Network Model


#### Background
You should be familiar with Python and basics of Data Analysis. You can refer to [kaggle learn](https://www.kaggle.com/learn/) and the machine [learning track](https://www.kaggle.com/learn/machine-learning).
Though a basic understanding of Natural Language Processing would be helpful, it's not a neccessity. This post will go through all the relevent information and provide required resources.

<br>
#### Pipeline Flow
***
![PipeLine Flow](./images/pipeline.png)

<br>
### DATA
***

Our approach to the project is to start with real time live datasets. For this we use Twitter API for fetching trends.


#### Data retreival

1. Fetching Twitter trends
2. Fetching tweets for trends and creating data for our model

***


#### Fetching Twitter Trends <br>
A trend on Twitter refers to a hashtag-driven topic that is immediately popular in a particular place at particular time. Trends help us analyse what are the themes that's being talked about most at any location at a given time.<br>


After registering with [Twitter Developers](https://developer.twitter.com/en/docs/basics/getting-started#get-started-app) we can access the API to fetch the latest trends at a location using the provided credentials. The API needs the [WOEID](http://www.woeidlookup.com/) of a location to fetch the trends of the place.<br>For processing the trends and making requests to the API we'll make use of Tweepy library.

#### Fetching tweets for trends <br>

Using the WOEID of a location and the credentials we can fetch the Twitter trends in the locations.
Following is the code snippet.

In [1]:
import tweepy
import csv
import pandas as pd

In [2]:
# Replace with correct credentials.
consumer_key = 'YOUR_CONSUMER_KEY_HERE'
consumer_secret = 'YOUR_CONSUMER_SECRET_HERE'
access_token = 'YOUR_APPLICATION_ACCESS_TOKEN_HERE'
access_token_secret = 'YOUR_APPLICATION_ACCESS_SECRET_HERE'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

Enter the WOEID for the required location.
The Twitter API endpoint trends_place is used to fetch the trends for the place


In [7]:
trends = api.trends_place(2367105)

[{'trends': [{'name': '#Save3Deckers', 'url': 'http://twitter.com/search?q=%23Save3Deckers', 'promoted_content': None, 'query': '%23Save3Deckers', 'tweet_volume': None}, {'name': '#DigiGirlzMA', 'url': 'http://twitter.com/search?q=%23DigiGirlzMA', 'promoted_content': None, 'query': '%23DigiGirlzMA', 'tweet_volume': None}, {'name': 'Brad Stevens', 'url': 'http://twitter.com/search?q=%22Brad+Stevens%22', 'promoted_content': None, 'query': '%22Brad+Stevens%22', 'tweet_volume': None}, {'name': '#NFLDraft', 'url': 'http://twitter.com/search?q=%23NFLDraft', 'promoted_content': None, 'query': '%23NFLDraft', 'tweet_volume': 613554}, {'name': 'Lamar Jackson', 'url': 'http://twitter.com/search?q=%22Lamar+Jackson%22', 'promoted_content': None, 'query': '%22Lamar+Jackson%22', 'tweet_volume': 147909}, {'name': '#InfinityWar', 'url': 'http://twitter.com/search?q=%23InfinityWar', 'promoted_content': None, 'query': '%23InfinityWar', 'tweet_volume': 356138}, {'name': '#Patriots', 'url': 'http://twitter

***
#### Fetching tweets for trends and creating data for our model
For each trend we fetch the tweets

In [11]:
tweepy.Cursor(api.search,q=trend,count=100,lang="en", since="2018-01-01")

Using the API end point we can fetch tweets for a trend and convert to a csv for processing.

Once we have the hashtags and tweets for the hashtags, we use Word2vec- a word embedding model for finding similarity between the user input and the tweets. 

<br>
### Recommend HashTag using Word2vec Model
***

### Word2vec

Word2Vec is a particularly computationally-efficient predictive model for learning word embeddings from raw text. It comes in two flavors, the Continuous Bag-of-Words model (CBOW) and the Skip-Gram model. These models are shallow, two-layer neural networks that are trained to reconstruct linguistic contexts of words. Word2Vec takes as input a large corpus of text and produces a vector space, typically of several hundred dimensions, with each unique word in the corpus being assigned a corresponding vector in the space. Word vectors are positioned in the vector space such that words that share common contexts in the corpus are in close proximity to one another in the space. 

We vectorize the corpus using Word2Vec word embeddings which gives us a 1000 dimensional vector for each word. The similarity between the entities is found using the cosine similarity between them. Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. We find out the nearest HashTag to the given user input/post from the Word2Vec model based on factors such as trends in the user's location, user's profile etc

![Word2vec Model](./images/word2vec.jpg)
![Word2vec Model](./images/king_queen.png)

***
### Code For HashTag Recommendation based on user's post

Importing required libraries

In [1]:
import pandas as pd
import seaborn as sns
%matplotlib inline
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
from  nltk.tokenize  import  TweetTokenizer 
import glob
import nltk
import string
import re
import pprint
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

C:\Users\dsain\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Reading all the csv files to create a corpus

In [17]:
allFiles = glob.glob("Data/*/*.csv", recursive=True)
frame = pd.DataFrame()

# To store filenames
filenames = []


for file_ in allFiles:    
    df = pd.read_csv(file_, encoding = "ISO-8859-1")
    filenames.append(file_.split("\\")[-1:][0].split(".")[0])
    frame = pd.concat([frame,df[df.columns[1]]])

Functions to preprocess the data by converting into lower case and removing stop words

In [18]:
def preprocessing(frame):
    frame = frame.apply(lambda x: x.astype(str).str.lower())
    frame = frame.apply(lambda x: x.astype(str).str.replace('b\'',''))
    frame = frame.apply(lambda x: x.astype(str).str.replace('b\"',''))
    return frame

#Function to build stopwords to remove them
def build_stopwords(): 
    stop_words = (stopwords.words('english'))

    list_remove = ['.','/','b\'','b\"','xe2','x80','xa6','x99t','x99s','x87','xb4','xaa','x9a','x9b','x93','xb2','x8c','x9c','xb5','xab']

    stop_words = stop_words +  word_tokenize(string.punctuation) + list_remove

    return stop_words

frame =  preprocessing(frame)

print(frame.head())



                                                   0
0  rt @wyomingpd: undercover #420day operations a...
1  rt @huffpost: on #420day, here's the exhaustiv...
2  @bevmedium @bri_clark love this! #truth #frees...
3  rt @deadpoolmovie: happy #420day everyone! let...
4   #420day rt @trevdon: happy birthday dogbert! ...


Removing stop words and building a corpus

In [19]:
corpus = []
tknzr  =  TweetTokenizer () 
stop_words = build_stopwords()

for row in frame[frame.columns[0]]:
#     Removing urls which can be noise
    row = re.sub(r'https?:\/\/.*[\r\n]*', '', row)
    word_tokens = tknzr.tokenize(row)
#     Removing stop words
    filtered_sentence = [w for w in word_tokens if not w in stop_words]    
    corpus.append(filtered_sentence)
    
print (corpus[:10])

[['rt', '@wyomingpd', 'undercover', '#420day', 'operations', 'place', 'n', 'nincognito', 'traps', 'set', 'throughout', 'city', 'n', 'n', '#happy420'], ['rt', '@huffpost', '#420day', "here's", 'exhaustive', 'list', 'everyone', "who's", 'died', 'marijuana', 'overdose', 'n', 'noh', 'wait', 'n', "nthere's", 'one'], ['@bevmedium', '@bri_clark', 'love', '#truth', '#freespirted', '#420day'], ['rt', '@deadpoolmovie', 'happy', '#420day', 'everyone', 'let', 'careful'], ['#420day', 'rt', '@trevdon', 'happy', 'birthday', 'dogbert', 'big', '8', 'even', 'though', 'day', 'dog', 'day', 'celebrated', 'today', 'extra', 'special'], ['rt', '@sueinrockville', '#420day', 'nnow', 'time', 'nrelease', 'anyone', 'jail', 'convicted', 'minor', 'possession', 'offense', 'nlegalize', 'medical', 'marijuana'], ['#420day', 'rt', '@sahluwal', 'nation', 'profits', 'sale', 'legalized', 'cannabis', 'want', 'remember', 'countless', 'americans', 'sitting', 'behind', 'bars'], ['rt', '@huffpost', '#420day', "here's", 'exhausti

Building a word2vec model

In [20]:
import timeit
start_time = timeit.default_timer()

model = Word2Vec(corpus, size=1000, window=3, min_count=1, workers=4)

elapsed = timeit.default_timer() - start_time

print(elapsed)

21.532314404504405


To validate the results from model

In [21]:
print('Number of words in vocab = ' + str(len(model.wv.vocab)))
# print('Similarity between woman and man = ' + str(model.wv.similarity('man', 'woman')))

Number of words in vocab = 39219


Function to get HashTag Suggestions

In [22]:
''' This function takes input post from the user and suggests hashtags which he can most likely relate his post with
    It recommends hashtags based on current trending topics and post content, 
    not just with the meaning of words but also the 
    context of the words is taken into consideration
'''
file_inputs = {}
def getHashTagSuggestion(input):
#     Tokenizing the input
    input = word_tokenize(input)
#     Getting a list of hashtags from the data collected

    for file in filenames:
        file = '#' + file.lower()
        file_inputs[file.lower()] = file
    
#     Iterating to find similaity between each words of input and all hashtags
    scores_out = []
    for out in file_inputs.keys():
        list_out = []
        if(out in model.wv.vocab):
            for word in input:    

                if(word in model.wv.vocab): 
                    list_out.append(model.similarity(word,out))
#                 print(model.similarity(word,out))

        if(len(list_out) > 0):
            scores_out.append((out,sum(list_out) / float(len(list_out))))
#     Returns sorted order of hashtag suggestion score
    scores_out = sorted(scores_out, key=lambda tup: tup[1],reverse = True)
    return scores_out

Input by user to recommend hashtags

In [27]:
input = 'I should read a book this night'

Displaying top HashTags related to user's post

In [28]:
HashTagSuggestion = []
suggestions = getHashTagSuggestion(input)
for out_tuple in suggestions[:5]:
    HashTagSuggestion.append(file_inputs[out_tuple[0]])
    
pprint.pprint (HashTagSuggestion)

['#hignfy',
 '#cosyreadingnight',
 '#thebutton',
 '#soldieringonawards',
 '#greatreadpbs']


## Finding out insights from the twitter posts:

The twitter posts can be analyzed to find out what are the most important entities in the posts. We have found out the most most frequently mentioned organizations, people and locations.
This information can be used to stay updated about the latest news.

***
### Code For Finding out most talked about person/organisation using Entity Tagging

Importing required libraries

In [29]:
import pandas as pd
import seaborn as sns
%matplotlib inline
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
from  nltk.tokenize  import  TweetTokenizer 
import glob
import nltk
import string
import re
import pprint
from nltk import word_tokenize, pos_tag, ne_chunk

Creating a pandas dataframe of the dataset

In [36]:
allFiles = glob.glob("Data/*/*.csv", recursive=True)
frame = pd.DataFrame()

filenames = []

for file_ in allFiles:
    
    df = pd.read_csv(file_, encoding = "ISO-8859-1")
    filenames.append(file_.split("\\")[-1:][0].split(".")[0])

    frame = pd.concat([frame,df[df.columns[1]]])

To check if the frame has been loaded successfully

In [37]:
print (frame.head())

                                                   0
0  b'RT @wyomingpd: Undercover #420day operations...
1  b"RT @HuffPost: On #420day, here's the exhaust...
2  b'@Bevmedium @Bri_Clark Love this! #truth #fre...
3  b'RT @deadpoolmovie: Happy #420Day everyone! L...
4  b' #420day RT @TrevDon: Happy Birthday Dogbert...


Example of how entity tagging gives results

In [38]:
sentence = "Tim Cook are working at Google facebook Apple."
 
print (ne_chunk(pos_tag(word_tokenize(sentence))))

(S
  (PERSON Tim/NNP)
  (ORGANIZATION Cook/NNP)
  are/VBP
  working/VBG
  at/IN
  (ORGANIZATION Google/NNP)
  facebook/NN
  Apple/NNP
  ./.)


Apply entity tagging model to entire corpus

In [39]:
ner = []
for row in frame.iterrows():
    ner.append(ne_chunk(pos_tag(word_tokenize(str(row[1])))))

Creating a dictionary of top topics to filter

In [40]:
top_topics = {}
for ner_entity in (ner):
    for entity in (ner_entity):
        if(type(entity) is nltk.tree.Tree):
            if (str(entity) not in top_topics.keys()):
                top_topics[str(entity)] = 1
            else: top_topics[str(entity)] = top_topics[str(entity)] + 1


Suggestion user's which are the most talked about companies,people and locations?

In [41]:
top_news_filter = []

for keys,values in top_topics.items():
    top_news_filter.append((keys,values))

top_news_filter_sorted = sorted(top_news_filter, key=lambda tup: tup[1],reverse = True)
top_news_filter_sorted[:20]

[('(ORGANIZATION FoodFri/NNP)', 1709),
 ('(ORGANIZATION EnoughIsEnough/NNP)', 1529),
 ('(ORGANIZATION NationalSchoolWalkout/NNP)', 1185),
 ('(ORGANIZATION EarthDay18/NNP)', 1130),
 ('(ORGANIZATION ALBOE/NNP)', 989),
 ('(ORGANIZATION RT/NNP)', 902),
 ('(ORGANIZATION FACup/NNP)', 854),
 ('(ORGANIZATION GreatReadPBS/NNP)', 801),
 ('(PERSON Wembley/NNP)', 746),
 ('(ORGANIZATION GaryVeeBreakfast/NNP)', 600),
 ('(ORGANIZATION HIGPA/NNP)', 519),
 ('(GPE Great/JJ)', 495),
 ('(ORGANIZATION SciFriLive/NNP)', 469),
 ('(ORGANIZATION American/JJ R/NNP)', 452),
 ('(ORGANIZATION NCAAGym/NNP)', 448),
 ('(ORGANIZATION MW18/NNP)', 421),
 ('(ORGANIZATION LFCU18s/NNP)', 419),
 ('(ORGANIZATION MUNTOT/NNP)', 399),
 ('(ORGANIZATION HelpCopyAFilm/NNP)', 368),
 ('(ORGANIZATION GardenersWorld/NNP)', 335)]

***
### Recommending Users whom you should follow based on your tweet/profiles

We can also use the tweet analysis to find tweets that are very similar to our input post. Figuring out the most similar tweet can be used to recommend the user to follow popular profiles based on the tweets from said popular profiles.


#### Spacy
We are going to use Spacy for analysing the post content and recommend similar posts and profiles. 

SpaCy is an open-source software library for advanced Natural Language Processing, written in the programming languages Python and Cython. It offers the fastest syntactic parser in the world. The library is published under the MIT license and currently offers statistical neural network models for English, German, Spanish, Portuguese, French, Italian, Dutch and multi-language NER, as well as tokenization for various other languages.


The reason for selecting SpaCy because it features neural models for tagging, parsing and entity recognition. A novel bloom embedding strategy with subword features is used to support huge vocabularies in tiny tables. Convolutional layers with residual connections, layer normalization and maxout non-linearity are used, giving much better efficiency than the standard models. \

We can use the SpaCy model to analyse the contents of a post. SpaCy contains out-of-the-box code for part-of-speech tagging. This helps us to tag parts of speech and analyse similarity based on what part of speech each word is and what role does it play in the context.

Using SpaCy we can identify tweets with similar content. This lets us measure the similarity by the role the words play in a sentence. SpaCy also provides named entity recognition to find out what entities are present in the corpus.

For our project we are going to focus on finding similarity between the post and all other posts across all tweets for the hashtags for every location. We can then compare the tweets to find similar ones and recommend the profile to the user using the named entity.


In [13]:
import pandas as pd
import seaborn as sns
%matplotlib inline
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
from  nltk.tokenize  import  TweetTokenizer 
import glob

c:\program files (x86)\python3.6.4\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Loading data into Dataframes in Pandas

Implementation of spacy and access to different properties is initiated by creating pipelines. A pipeline is created by loading the models. There are different type of models provided in the package which contains the information about language – vocabularies, trained vectors, syntaxes and entities. We will load the default model which is english-core-web.

The object “nlp” is used to create documents, access linguistic annotations and different nlp properties. We create a document by loading the files and the data in our pipeline.

In [21]:
import spacy

spacy.load("en")
import en_core_web_sm
nlp = en_core_web_sm.load()
frames = []
frame_words = []

location_df = pd.DataFrame()
locationFiles = glob.glob("Data/*/*.csv", recursive=True)


Fetch all the hashtags and their tweets and load create corpus.



In [22]:

for file in locationFiles:
    df = pd.read_csv(file, encoding = "ISO-8859-1")
    location_df = pd.concat([location_df,df[df.columns[1]]])

for index,frame in location_df.iterrows():
    frames.append(nlp(frame[0]))
    frame_words.append(frame[0])


We need the import the operator module for operations on intrinsic functions.

In [23]:
import operator


We can test out the code against a user text. Lets use a real example of a tweet. Following is a tweet that was tweeted on Twitter.

In [32]:
input = "To Wembley for Final. Wht a match!!"

user_input_nlp = nlp(input)
mostsimilarword = ""
mostsimilarword_similarity = 0
reference = {}
index = 0

Let us now find the tweets that are similar to the user tweet.

We use the document1.similarity(document2) provided by SpaCy to find cosine similairty between 2 documents.
By doing the similarity between all the tweets we can find out how each tweet is related to every other tweet.
We then find out the top most similar tweet. In order to recommend a popular profile that had tweeted a similar tweet we check if the similar tweet was in context a retweet of the popular tweet.

If it is we know that a profile meets the rules set for a popular profile to be recommended.

In [33]:
for idx, doc in enumerate(frames):
    similarity_of_word = doc.similarity(user_input_nlp)
    if (similarity_of_word > mostsimilarword_similarity):
        print(doc)
        similar_tweet = frame_words[idx]
        if "@" in similar_tweet:
            mostsimilarword_similarity = similarity_of_word
            mostsimilarword = similar_tweet
            reference = set([i[1:].rstrip(':') for i in similar_tweet.split() if i.startswith("@")])




b' #FACup RT @D_DeGea: Wembley is so special. Amazing atmosphere, great game... and through to the final!!!! \xf0\x9f\x92\xaa\xf0\x9f\x8f\xbb\xf0\x9f\x94\xb4\n\nWembley es especial. Gran ambiente,\xe2\x80\xa6'
b'BREAKING! Davide Zappacosta is set to start instead of Emerson Palmieri against Southampton tomorrow, acc to Daily Mail. #CFC #FACup'
b'RT @rioferdy5: Boooom... great work from Pogba &amp; caressed header from Sanchez!!!! Come on #MUFC \xe2\x9a\xbd\xef\xb8\x8f #FACup https://t.co/4YFSjoO0Ii'
b'@AnderHerrera @ManUtd The true Man of the Match !! #FACup #MUNTOT'
b'What a day at Wembley. Onto the Final. \xf0\x9f\x9a\x80\xf0\x9f\x9a\x80 #MUFC #FACup'


In [34]:
print("Similar Tweet: " + mostsimilarword)

top_reference = reference.pop()

print("\n You might like to befriend: ")
print("@"+top_reference)

Similar Tweet: b'@AnderHerrera @ManUtd The true Man of the Match !! #FACup #MUNTOT'

 You might like to befriend: 
@ManUtd


#### Using Tweepy to fetch the details about the popular profile and recommend the profile to the user.

In [36]:
consumer_key = 'YOUR_CONSUMER_KEY_HERE'
consumer_secret = 'YOUR_CONSUMER_SECRET_HERE'
access_token = 'YOUR_APPLICATION_ACCESS_TOKEN_HERE'
access_token_secret = 'YOUR_APPLICATION_ACCESS_SECRET_HERE'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [37]:
stuff = api.user_timeline(screen_name = top_reference, count = 100, include_rts = True, exclude_replies=True)
print (stuff)

[Status(_api=<tweepy.api.API object at 0x000002ADB071ACC0>, _json={'created_at': 'Fri Apr 27 18:38:27 +0000 2018', 'id': 989936831624953856, 'id_str': '989936831624953856', 'text': "Heart-warming scenes at yesterday's #MUDreamDay! ♥️ https://t.co/07o3U9nSjT", 'truncated': False, 'entities': {'hashtags': [{'text': 'MUDreamDay', 'indices': [36, 47]}], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 989936499960365057, 'id_str': '989936499960365057', 'indices': [52, 75], 'media_url': 'http://pbs.twimg.com/media/Dbz2TBLVQAAgfnU.jpg', 'media_url_https': 'https://pbs.twimg.com/media/Dbz2TBLVQAAgfnU.jpg', 'url': 'https://t.co/07o3U9nSjT', 'display_url': 'pic.twitter.com/07o3U9nSjT', 'expanded_url': 'https://twitter.com/ManUtd/status/989936831624953856/video/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 1200, 'h': 675, 'resize': 'fit'}, 'small': {'w': 680, 'h': 383, 'resize': 'fit'}, 'large': {'w': 1280, 'h': 720, 'resize': 'fi

In [38]:
#for status in stuff:
status = stuff[0]
print ("User Name:" + status.user.name)
print ("User Status:" + status.user.description)
if(status.user.location): 
    print ("User Location:" + status.user.location)
    
print (status.user.profile_banner_url)

User Name:Manchester United
User Status:Official account. Now playing on https://t.co/Y84feSQO1P.
User Location:Manchester, England
https://pbs.twimg.com/profile_banners/558797310/1524470396


In [39]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= status.user.profile_banner_url)

The text in the document by Rohan Jahagirdar, Sai Nikhil Dogiparty is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/

The code in the document by Rohan Jahagirdar, Sai Nikhil Dogiparty is licensed under the MIT License https://opensource.org/licenses/MIT

All contents of the tweets includind the profile name, posts, the word "tweet", images all other contents are copyright of their respective owners and of [Twitter](https://twitter.com)


![License](https://licensebuttons.net/l/by/3.0/us/88x31.png)


